In [7]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import pandas as pd
import socket
from urllib3.connection import HTTPConnection
import matplotlib.pyplot as plt
import seaborn as sns
from lxml import html
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

In [8]:
HTTPConnection.default_socket_options = (
    HTTPConnection.default_socket_options + [
        (socket.SOL_SOCKET, socket.SO_KEEPALIVE, 1),
        (socket.SOL_TCP, socket.TCP_KEEPIDLE, 45),
        (socket.SOL_TCP, socket.TCP_KEEPINTVL, 10),
        (socket.SOL_TCP, socket.TCP_KEEPCNT, 6)
    ] )

In [9]:
tipomoneda_vivienda=[]
precio_vivienda=[]
direccion_vivienda=[]
metros_cuadrados_vivienda=[]
años_vivienda=[]
ambientes_vivienda=[]

link_inmobiliaria=[]



for i in range(0,159):
    
    if i==0:
        link = 'https://www.argenprop.com/departamento-alquiler-localidad-capital-federal'
        
    else:
        link= 'https://www.argenprop.com/departamento-alquiler-localidad-capital-federal'+str("-pagina-{}".format(i))
        


    driver = webdriver.Chrome()



    driver.get(link)

    time.sleep(15)

    html = driver.page_source

    driver.implicitly_wait(20)

    soup= BeautifulSoup(html,'html.parser')

    seleccionador_precio =soup.find_all('p',class_='card__price')

    tipo_moneda=[]
    precio=[]
    direccion=[]
    for currency in seleccionador_precio:
        try:
            #SELECCION DE TIPO DE MONEDA
            #SELECCION DE PRECIO
            moneda=currency.find('span',class_='card__currency').text
            tipo_moneda.append(moneda)
            valor=currency.text
            precio.append(valor)
        except:
            tipo_moneda.append('faltante')
        
            precio.append('faltante')

    #PRIMER LOOP PARA ENCONTRAR EL TAG
    #SEGUNDO LOOP PARA AGARRAR EL SRC
    #
    img_l = soup.find_all('div',class_='card__agent')

    link=[]

    for i in img_l:
        if i.img== None:
            link.append('no hay')
        else:

            img_ll=i.img
            link.append(img_ll)

    link_src=[]
    for src in link:
        if src== 'no hay':
            link_src.append('No aparece Inmobiliaria')
        else:
        
            link_src.append(src['src'])

    ################################################################3
    
    seleccionador_ubicacion = soup.find_all('div',class_='card__monetary-values')

    for i in seleccionador_ubicacion:
        try:
            lugar=i.find('h2').text
            direccion.append(lugar)
        except:
            direccion.append('falta lugar')

    metros_cuadrados =[]
    años=[]
    ambientes=[]
    precisiones = soup.find_all('ul',class_='card__main-features')

    for precision in precisiones:
        try:
            #metros cuadrados
            sup_cub = precision.find('li').span.text
            metros_cuadrados.append(sup_cub)

            #ambientes
            dormitorios =precision.select('li')[1].span.text
            ambientes.append(dormitorios)

            #años
            tiempo = precision.select('li')[2].span.text
            años.append(tiempo)

        except:
            #metros cuadrados
            metros_cuadrados.append('Faltante')

            #ambiente
            ambientes.append('Faltante')

            #años
            años.append('Faltante')

    
    tipomoneda_vivienda.append(tipo_moneda)
    precio_vivienda.append(precio)
    direccion_vivienda.append(direccion)
    metros_cuadrados_vivienda.append(metros_cuadrados)
    años_vivienda.append(años)
    ambientes_vivienda.append(ambientes)
    link_inmobiliaria.append(link_src)
    
    driver.quit()

    
    

In [11]:
print(len(tipomoneda_vivienda))
print(len(precio_vivienda))
print(len(direccion_vivienda))
print(len(metros_cuadrados_vivienda))
print(len(años_vivienda))
print(len(ambientes_vivienda))
print(len(link_inmobiliaria))

159
159
159
159
159
159
159


106

In [12]:
df =pd.DataFrame({'tipo_moneda':tipomoneda_vivienda,'precio_vivienda':precio_vivienda,
                'direccion_vivienda':direccion_vivienda,'metros_cuadrados':metros_cuadrados_vivienda,
                'años_vivenda':años_vivienda,'ambientes':ambientes_vivienda,'inmobiliaria':link_inmobiliaria})

In [14]:
df.to_csv('argenprop.csv',index=False)